In [1]:
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import cudf
import cuml
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.model_selection import train_test_split
from scipy.stats import randint
from scipy.stats import randint
import os
import pickle

In [5]:
retrain = False

if not retrain:
    rf_path = r'/mnt/c/users/attic/hls_kelp/random_forest/cu_rf'
    with open(rf_path, 'rb') as f:
        cu_rf = pickle.load(f)
classified_path = r'/mnt/c/users/attic/hls_kelp/imagery/rf_classified_cuML'
unclassified_path = r'/mnt/c/users/attic/hls_kelp/imagery/rf_prepped'
files = os.listdir(unclassified_path)

In [6]:
for i, file in enumerate(files):
    if os.path.isfile(os.path.join(classified_path, f'{file}_kelp_classified.tif')):
        continue
    with rasterio.open(os.path.join(unclassified_path,file)) as img:
        file_img = img.read(indexes=[1, 2, 3, 4, 5, 6])
        width,height = img.shape
        img_data= file_img.reshape(file_img.shape[0], -1).T
        img_data = img_data.astype('float32')
        kelp_data = cu_rf.predict(img_data)
        kelp_img = kelp_data.reshape(width, height)
        data_type = rasterio.uint8
        profile = {
            'driver': 'GTiff',
            'width': width,
            'height': height,
            'count': 1,  # one band
            'dtype': data_type,  # assuming binary mask, adjust dtype if needed
            'crs': img.crs,
            'transform': img.transform,
            'nodata': 0  # assuming no data is 0
        }
        # Write the land mask array to GeoTIFF
        with rasterio.open(os.path.join(classified_path, f'{file}_kelp_classified.tif'), 'w', **profile) as dst:
            dst.write(kelp_img.astype(data_type), 1)
        print(f'{i+1} / {len(files)}')
        

1 / 31
2 / 31
3 / 31
4 / 31
5 / 31
6 / 31
7 / 31
8 / 31
9 / 31
10 / 31
11 / 31
12 / 31
13 / 31
14 / 31
15 / 31
16 / 31
17 / 31
18 / 31
19 / 31
20 / 31
21 / 31
22 / 31
23 / 31
24 / 31
